In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import os, pathlib

## Training a convnet from scratch on a small dataset

### The relevance of deep learning for small-data problems

### Downloading the data

-- from Kaggle

In [2]:
from google.colab import files
files.upload()    # you will chose file for your kaggle.json, downloaded from laggle/settings

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tariqjamil","key":"b193509d39c88c44ca89e3cd572962cb"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
#!kaggle competitions download -c dogs-vs-cats
#!kaggle datasets download -d cdart99/food20dataset
!kaggle datasets download -d utkarshsaxenadn/fast-food-classification-dataset

100% 818M/821M [00:10<00:00, 137MB/s]
100% 821M/821M [00:10<00:00, 78.9MB/s]


The Data Extraction

In [5]:
!unzip -qq /content/fast-food-classification-dataset.zip

In [ ]:
#!unzip -qq train.zip

**Copying images to training, validation, and test directories**

In [ ]:
%%script true

import os, shutil, pathlib

original_dir = pathlib.Path("train")   # has 2 subfolders cat and dog

new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

In [ ]:
#trn = os.listdir(original_dir)
#len(trn)

25000

In [ ]:
%%script true

!rm -rf 'cats_vs_dogs_small'
make_subset(subset_name="train",      start_index=0,    end_index=2000)
make_subset(subset_name="validation", start_index=2000, end_index=2500)
make_subset(subset_name="test",       start_index=2500,end_index=3500)

In [6]:
base_dir = pathlib.Path('/content/Fast Food Classification V2')

In [ ]:
#train_list = os.listdir(base_dir/'train_set')
test_list = os.listdir(base_dir/'test_set')

In [ ]:
for root, _, files in os.walk(base_dir):
  print(subfolder)
  for f in files:
      fpath = os.path.join(root, f)


In [ ]:
class_names = []
for f in base_dir.rglob('*'):
  if f.is_dir():
    class_names.append(str(f).rsplit('/')[-1])

class_names = class_names[2:]

In [ ]:
class_names

['noodles',
 'biriyani',
 'samosa',
 'gulab jamun',
 'dhokla',
 'kathi roll',
 'tandoori chicken',
 'idly',
 'meduvadai',
 'chaat',
 'ven pongal',
 'dosa',
 'vada pav',
 'upma',
 'poori',
 'butternaan',
 'halwa',
 'paniyaram',
 'bisibelebath',
 'chappati',
 'noodles',
 'biriyani',
 'samosa',
 'gulab jamun',
 'dhokla',
 'kathi roll',
 'tandoori chicken',
 'idly',
 'meduvadai',
 'chaat',
 'ven pongal',
 'dosa',
 'vada pav',
 'upma',
 'poori',
 'butternaan',
 'halwa',
 'paniyaram',
 'bisibelebath',
 'chappati']

### Building the model

**Instantiating a small convnet for dogs vs. cats classification**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 89, 89, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                        

**Configuring the model for training**

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

### Data preprocessing

**Using `image_dataset_from_directory` to read images**

In [10]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    base_dir / "Train",
    image_size=(180, 180),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
    base_dir / "Valid",
    image_size=(180, 180),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    base_dir / "Test",
    image_size=(180, 180),
    batch_size=32)

Found 15000 files belonging to 10 classes.
Found 3500 files belonging to 10 classes.
Found 1500 files belonging to 10 classes.


**Displaying the shapes of the data and labels yielded by the `Dataset`**

In [ ]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

InvalidArgumentError: ignored

**Fitting the model using a `Dataset`**

In [ ]:
cb_check_pointing = keras.callbacks.ModelCheckpoint(
    filepath="convnet_from_scratch.keras",
    save_best_only=True,
    monitor="val_loss")

cb_early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=0,
    mode='auto',
    restore_best_weights=True,
    start_from_epoch=0)

callbacks = [cb_early_stop]#, cb_check_pointing]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 18s 54ms/step - loss: 0.6994 - accuracy: 0.5167 - val_loss: 0.6885 - val_accuracy: 0.5080
Epoch 2/30
125/125 [==============================] - 8s 63ms/step - loss: 0.6778 - accuracy: 0.5980 - val_loss: 0.6605 - val_accuracy: 0.5770
Epoch 3/30
125/125 [==============================] - 8s 62ms/step - loss: 0.6424 - accuracy: 0.6410 - val_loss: 0.6429 - val_accuracy: 0.6300
Epoch 4/30
125/125 [==============================] - 7s 51ms/step - loss: 0.5905 - accuracy: 0.6880 - val_loss: 0.5481 - val_accuracy: 0.7480
Epoch 5/30
106/125 [========================>.....] - ETA: 1s - loss: 0.5500 - accuracy: 0.7211

**Displaying curves of loss and accuracy during training**

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
model.save('convnet_from_scratch.keras')

In [ ]:
from keras.saving import load_model

test_model = load_model('convnet_from_scratch.keras')

**Evaluating the model on the test set**

In [ ]:
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Using data augmentation

**Define a data augmentation stage to add to an image model**

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

**Displaying some randomly augmented training images**

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

**Defining a new convnet that includes image augmentation and dropout**

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

**Training the regularized convnet**

In [ ]:
cb_check_pointing = keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.tf",   # .keras gives error
        save_best_only=True,
        monitor="val_loss")

callbacks = [cb_check_pointing, cb_early_stop]

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Evaluating the model on the test set**

In [ ]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.tf")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

## Leveraging a pretrained model

### Feature extraction with a pretrained model

**Instantiating the VGG16 convolutional base**

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

In [ ]:
conv_base.summary()

#### Fast feature extraction without data augmentation

**Extracting the VGG16 features and corresponding labels**

In [ ]:
import numpy as np

def get_features_and_labels(dataset):

    all_features, all_labels = [], []

    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)

    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features,   val_labels   =  get_features_and_labels(validation_dataset)
test_features,  test_labels  =  get_features_and_labels(test_dataset)

In [ ]:
train_features.shape

**Defining and training the densely connected classifier**

In [ ]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.tf",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

**Plotting the results**

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

#### Feature extraction together with data augmentation

**Instantiating and freezing the VGG16 convolutional base**

In [ ]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)

conv_base.trainable = False

**Printing the list of trainable weights before and after freezing**

In [ ]:
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

**Adding a data augmentation stage and a classifier to the convolutional base**

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.tf",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Evaluating the model on the test set**

In [ ]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Fine-tuning a pretrained model

In [ ]:
conv_base.summary()

**Freezing all layers until the fourth from the last**

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

**Fine-tuning the model**

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

## Summary